In [49]:
import keras.backend as K
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

import cnn_builder as cbuild
import cnn_methods as cfunc
import config
import csv
import helper_fxns as hf
import importlib
import matplotlib.pyplot as plt
import numpy as np
import operator
import os
import pandas as pd
import random
%matplotlib inline

In [141]:
importlib.reload(config)
importlib.reload(cfunc)
importlib.reload(hf)
importlib.reload(cbuild)
C = config.Config()

In [142]:
model = cbuild.build_cnn(C, Adam(lr=0.02))#'adam'
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_125 (InputLayer)           (None, 36, 36, 12, 1) 0                                            
____________________________________________________________________________________________________
input_126 (InputLayer)           (None, 36, 36, 12, 1) 0                                            
____________________________________________________________________________________________________
input_127 (InputLayer)           (None, 36, 36, 12, 1) 0                                            
____________________________________________________________________________________________________
conv3d_104 (Conv3D)              (None, 36, 36, 12, 64 1216        input_125[0][0]                  
___________________________________________________________________________________________

model = cbuild.run_cnn(model, C)

In [143]:
nb_classes = len(C.classes_to_include)
voi_df = pd.read_csv(C.art_voi_path)
orig_data_dict, num_samples = cfunc.collect_unaug_data(C, voi_df)
print(num_samples)

avg_X2 = {}
for cls in C.classes_to_include:
    avg_X2[cls] = np.mean(orig_data_dict[cls][1], axis=0)

{'hemangioma': 56, 'cholangio': 56, 'cyst': 75, 'colorectal': 147, 'hcc': 87, 'fnh': 61}


In [144]:
train_ids = {} #filenames of training set originals
test_ids = {} #filenames of test set
X_test = []
X2_test = []
Y_test = []
Z_test = []
X_train_orig = []
X2_train_orig = []
Y_train_orig = []
Z_train_orig = []

train_samples = {}

for cls_num, cls in enumerate(orig_data_dict):
    cls_num = C.classes_to_include.index(cls)

    train_samples[cls] = round(num_samples[cls]*C.train_frac)

    order = np.random.permutation(list(range(num_samples[cls])))
    train_ids[cls] = list(orig_data_dict[cls][2][order[:train_samples[cls]]])
    test_ids[cls] = list(orig_data_dict[cls][2][order[train_samples[cls]:]])

    X_test = X_test + list(orig_data_dict[cls][0][order[train_samples[cls]:]])
    X2_test = X2_test + list(orig_data_dict[cls][1][order[train_samples[cls]:]])
    Y_test = Y_test + [[0] * cls_num + [1] + [0] * (nb_classes - cls_num - 1)] * \
                        (num_samples[cls] - train_samples[cls])
    Z_test = Z_test + test_ids[cls]

    X_train_orig = X_train_orig + list(orig_data_dict[cls][0][order[:train_samples[cls]]])
    X2_train_orig = X2_train_orig + list(orig_data_dict[cls][1][order[:train_samples[cls]]])
    Y_train_orig = Y_train_orig + [[0] * cls_num + [1] + [0] * (nb_classes - cls_num - 1)] * \
                        (train_samples[cls])
    Z_train_orig = Z_train_orig + train_ids[cls]

    print("%s has %d samples for training (%d after augmentation) and %d for testing" %
          (cls, train_samples[cls], train_samples[cls] * C.aug_factor, num_samples[cls] - train_samples[cls]))

#Y_test = np_utils.to_categorical(Y_test, nb_classes)
#Y_train_orig = np_utils.to_categorical(Y_train_orig, nb_classes)
X_test = [np.array(X_test), np.array(X2_test)]
X_train_orig = [np.array(X_train_orig), np.array(X2_train_orig)]

Y_test = np.array(Y_test)
Y_train_orig = np.array(Y_train_orig)

Z_test = np.array(Z_test)
Z_train_orig = np.array(Z_train_orig)

X_test = cfunc.separate_phases(X_test)
X_train_orig = cfunc.separate_phases(X_train_orig)

hemangioma has 45 samples for training (2250 after augmentation) and 11 for testing
cholangio has 45 samples for training (2250 after augmentation) and 11 for testing
cyst has 60 samples for training (3000 after augmentation) and 15 for testing
colorectal has 118 samples for training (5900 after augmentation) and 29 for testing
hcc has 70 samples for training (3500 after augmentation) and 17 for testing
fnh has 49 samples for training (2450 after augmentation) and 12 for testing


train_generator = train_generator_func()
model_pretrain.fit_generator(train_generator, steps_per_epoch=120, epochs=50)#, callbacks=[early_stopping])

In [136]:
#early_stopping = EarlyStopping(monitor='acc', min_delta=0.01, patience=4)
train_generator = cbuild.train_generator_func(C, train_ids, intensity_df, voi_df, avg_X2, n=5, n_art=5)
model.fit_generator(train_generator, steps_per_epoch=50, epochs=15)#, callbacks=[early_stopping])

Epoch 1/100
50/50 [==============================] - 23s - loss: 1.1761 - acc: 0.5870    
Epoch 2/100
50/50 [==============================] - 22s - loss: 0.7637 - acc: 0.7373    
Epoch 3/100
50/50 [==============================] - 22s - loss: 0.5922 - acc: 0.8047    
Epoch 4/100
50/50 [==============================] - 22s - loss: 0.4861 - acc: 0.8283    
Epoch 5/100
50/50 [==============================] - 22s - loss: 0.4201 - acc: 0.8503    
Epoch 6/100
50/50 [==============================] - 22s - loss: 0.3705 - acc: 0.8753    
Epoch 7/100
50/50 [==============================] - 22s - loss: 0.3318 - acc: 0.8923    
Epoch 8/100
50/50 [==============================] - 22s - loss: 0.2993 - acc: 0.8973    
Epoch 9/100
50/50 [==============================] - 21s - loss: 0.2745 - acc: 0.9123    
Epoch 10/100
50/50 [==============================] - 21s - loss: 0.2171 - acc: 0.9330    
Epoch 11/100
 4/50 [=>............................] - ETA: 20s - loss: 0.2090 - acc: 0.9542

KeyboardInterrupt: 

hist = model.fit(X_train2, Y_train2, batch_size=32, epochs=200, validation_data=(X_test, Y_test), callbacks=[early_stopping])

In [92]:
voi_df_art = pd.read_csv(C.art_voi_path)
voi_df_ven = pd.read_csv(C.ven_voi_path)
voi_df_eq = pd.read_csv(C.eq_voi_path)

with open(C.small_voi_path, 'r') as csv_file:
    reader = csv.reader(csv_file)
    small_vois = dict(reader)
for key in small_vois:
    small_vois[key] = [int(x) for x in small_vois[key][1:-1].split(', ')]

In [ ]:
cbuild.overnight_run(C)

In [ ]:
C.output_img_dir = "Z:\\OUTPUT\\12-08-2d-bn"
C.classes_to_include

In [137]:
#plot_with_bbox(fn_list[2], cls_mapping[wrong_guesses[2]])
Y_pred = model.predict(X_test)
y_true = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_test])
y_pred = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_pred])

cm = confusion_matrix(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="weighted")

#cfunc.save_output(Z_test, y_pred, y_true, voi_df_art, small_vois, C.classes_to_include, C)

In [133]:
Y_pred = model.predict(X_train_orig)
y_true = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_train_orig])
y_pred = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_pred])

cm = confusion_matrix(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="weighted")

#cfunc.save_output(Z_train_orig, y_pred, y_true, voi_df_art, small_vois, C.classes_to_include, C, save_dir="Z:\\OUTPUT\\12-08-3d\\training")

In [62]:
cm

array([[14,  0,  0,  0,  0,  3],
       [ 0,  6,  3,  0,  2,  0],
       [ 0,  2, 12,  0,  1,  0],
       [ 0,  0,  1, 14,  0,  0],
       [ 0,  0,  1,  3,  7,  0],
       [ 2,  0,  0,  0,  0, 10]], dtype=int64)

In [123]:
y_true_simp, y_pred_simp, _ = cfunc.condense_cm(y_true, y_pred, C.classes_to_include)

In [124]:
accuracy_score(y_true_simp, y_pred_simp)

0.83157894736842108

In [138]:
accuracy_score(y_true, y_pred)

0.58947368421052626

4 cls: 94/94%, 86/88%, 86%/90%
5 cls: 80%/89%, 72%/86%, 84%/86%

6 cls no artificial, separate phases: 75%/84%

2d: 77%/84%, 74%/80%
3d: 78%/89%
2d bn: 77%/82%, 
3d bn: 84%/87%, 74%/83%
3d bn no artif: 84%/87%, 74%/83%

fn_list = Z_test[~np.equal(y_pred, y_true)]
wrong_guesses = np.array(y_pred)[~np.equal(y_pred, y_true)]

activ = model2.predict(X_train)
#activ = model2.predict(np.expand_dims(X_train[10],axis=0))

import operator
#print("Ground truth:", [C.classes_to_include[max(enumerate(x), key=operator.itemgetter(1))[0]] for x in Y_val[::30]])
Y_ = model.predict(X_test)
print("Predictions:", [C.classes_to_include[max(enumerate(x), key=operator.itemgetter(1))[0]] + " (%.5f%% probability)" % (max(x)*100) for x in Y_])